In [1]:
!pip3 install pandas

In [3]:
!wget https://db.idrblab.net/ttd/sites/default/files/ttd_database/P1-06-Target_disease.txt -O P1-06-Target_disease.txt

--2024-04-16 12:18:12--  https://db.idrblab.net/ttd/sites/default/files/ttd_database/P1-06-Target_disease.txt
Resolving db.idrblab.net (db.idrblab.net)... 47.88.56.212
Connecting to db.idrblab.net (db.idrblab.net)|47.88.56.212|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 891227 (870K) [text/plain]
Saving to: ‘P1-06-Target_disease.txt’

P1-06-Target_diseas 100%[===================>] 870.34K  2.04MB/s    in 0.4s    

2024-04-16 12:18:13 (2.04 MB/s) - ‘P1-06-Target_disease.txt’ saved [891227/891227]



In [4]:
import pandas as pd

# 步骤1：读取数据
file_path = 'P1-06-Target_disease.txt'  # 你的文件路径
with open(file_path, 'r') as file:
    for i in range(0, 22):
        next(file)

    data_sections = file.read().split('\n\n')  # 按空行分割各个section

# 步骤2：解析数据
data_records = []
for section in data_sections:
    record = {}
    lines = section.split('\n')
    for line in lines:
        parts = line.split('\t')
        if len(parts) < 3:  # 检查是否有足够的部分
            continue  # 跳过这一行

        key = parts[1]
        value = parts[2]
        
        if key == 'TARGETID':
            record['source_id'] = value
            record['ttd_source_id'] = value
        elif key == 'TARGNAME':
            record['source_name'] = value
        elif key == 'INDICATI':
            # 提取并处理ICD id，确保没有多余的空格
            icd_id = ' '.join(parts[4:]).split('[')[-1].strip(']')
            record['target_id'] = icd_id.replace(' ', '')
            record['target_name'] = parts[3]
            record['ttd_target_id'] = icd_id.replace(' ', '')
            
    # 添加固定值
    record['source_type'] = 'Gene'
    record['target_type'] = 'Disease'
    record['relation_type'] = 'DRUGBANK::treats::Compound:Disease'
    record['resource'] = 'TTD'
    
    if record:  # 确保记录是非空的
        data_records.append(record)

# 步骤3：构建DataFrame
df = pd.DataFrame(data_records)

# 步骤4：导出到CSV
output_file_path = 'P1-06-Target_disease_extracted.csv'  # 输出文件的路径
df.to_csv(output_file_path, index=False)

print("Data processing completed and the csv file has been saved.")


Data processing completed and the csv file has been saved.


In [5]:
import pandas as pd

df1 = pd.read_csv('P1-06-Target_disease_extracted.csv')
df2 = pd.read_csv('../idmapping/ttd_gene_id.tsv', sep='\t')

df2.rename(columns={'target_id': 'gene_symbol', 'ttd_target_id': 'ttd_gene_id'}, inplace=True)
print(df2.head())
merged_df = pd.merge(
    df1,
    df2[["ttd_gene_id", "gene_symbol"]],
    left_on="source_id",
    right_on="ttd_gene_id",
    how="left",
)

merged_df.drop("ttd_gene_id", axis=1, inplace=True)
merged_df["source_name"] = merged_df["gene_symbol"].str.split(":").str[1]
merged_df["source_id"] = merged_df["gene_symbol"]
merged_df.drop("gene_symbol", axis=1, inplace=True)
invalid_df = merged_df[merged_df["source_id"].isnull() | merged_df["target_id"].isnull()]
merged_df = merged_df[merged_df["source_id"].notnull() & merged_df["target_id"].notnull()]

invalid_df.to_csv("invalid_ttd_target_disease.tsv", index=False, sep="\t")
merged_df.to_csv("formatted_ttd_target_disease.tsv", index=False, sep="\t")

  ttd_gene_id                                  target_name   gene_symbol  \
0      T47101  Fibroblast growth factor receptor 1 (FGFR1)  SYMBOL:FGFR1   
1      T59328      Epidermal growth factor receptor (EGFR)   SYMBOL:EGFR   
2      T89515                Polypeptide deformylase (PDF)    SYMBOL:PDF   
3      T08391                       Janus kinase 2 (JAK-2)   SYMBOL:JAK2   
4      T07663                 Phosphodiesterase 5A (PDE5A)  SYMBOL:PDE5A   

  ttd_uniprot_id target_type  \
0    FGFR1_HUMAN  Successful   
1     EGFR_HUMAN  Successful   
2     DEFM_HUMAN  Successful   
3     JAK2_HUMAN  Successful   
4    PDE5A_HUMAN  Successful   

                                            synonyms  \
0  c-fgr; bFGF-R-1; bFGF-R; N-sam; HBGFR; Fms-lik...   
1  Receptor tyrosine-protein kinase erbB-1; Proto...   
2                                                PDF   
3                       Tyrosine-protein kinase JAK2   
4  cGMP-specific 3',5'-cyclic phosphodiesterase; ...   

             